In [1]:
def NaiveBayes():
    trainList = []
    testList = []
    classes = {"spam", "ham"}
    os.chdir("data")
    for dataset in ["dataset 1", "dataset 2", "dataset 3"]:
        os.chdir(dataset)
        for folder in ["train", "test"]:
            os.chdir(folder)
            for classfication in classes:
                os.chdir(classfication)
                for file in os.listdir():
                    filename = os.fsdecode(file)
                    if filename.endswith(".txt"):
                        with open(filename, encoding='utf8', errors='ignore') as f:
                            if folder == "train":
                                trainList.append((f.read(), classfication))
                            elif folder == "test":
                                testList.append((f.read(), classfication))
                os.chdir('..')
            os.chdir('..')
        print(dataset.capitalize() + "----------------------")
        print("The accuracy for " + dataset + " is " + str(Accuracy(classes, trainList, testList)))
        trainList.clear()
        testList.clear()
        os.chdir('..')
    os.chdir('..')

In [ ]:
def TrainMultinomialNB(setClasses, trainingSet):
    print("Data initialized:")
    print("Stared training...")
    V = ExtractVocabulary(trainingSet)
    N = CountDocs(trainingSet)
    prior ={}
    condprob = collections.defaultdict(dict)
    TctT = 0
    for c in setClasses:
        Nc = CountDocsInClass(trainingSet, c)
        prior[c] = Nc/N
        textC = ConcatenateTextOfAllDocsInClass(trainingSet, c)
        for t in V:
            Tct = CountTokensofTerm(textC, t)
            if TctT == 0:
                for tp in V:
                    TctT = TctT + CountTokensofTerm(textC, tp) + 1
            condprob[t][c] = (Tct + 1)/TctT
    return V, prior, condprob

In [ ]:
def ApplyMultinomialNB(setClass, Vocab, PriorKnow, Condprobs, testData):
    W = ExtractTokensFromDoc(Vocab, testData[0])
    score = {}
    for c in setClass:
        score[c] = math.log(PriorKnow[c])
        for t in W:
            score[c] += math.log(Condprobs[t][c])
    return max(score, key=score.get)

In [ ]:
Finding the accuracy 

In [ ]:
def Accuracy(setClasses, trainingSet, testSet):
    #print("Initializing trainNB")
    trainNB = TrainMultinomialNB(setClasses, trainingSet)
    correctClass = 0
    for instance in testSet:
        #print("Finding classification of testSet")
        if instance[1] == ApplyMultinomialNB(setClasses, trainNB[0], trainNB[1], trainNB[2], instance):
            correctClass += 1
    #print("Found accuracy")
    acc = correctClass/CountDocs(testSet)
    return acc